# Scraping Process  

1. 웹 페이지 크롤링 : fetch(url)  
    url 주소의 웹페이지를 추출하여 문자열 decoding 하여 html return 
2. 스크래핑 : scrape(html)   
    html에서 정규표현식을 이용해 정보 추출
3. 데이터 저장 : save(db_path, data)  
    추출된 data를 db_path로 지정된 DB에 저장

## 도서 목록 저장 프로세스 

In [6]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

### 1. 웹 페이지 크롤링

In [7]:
def fetch(url):
    
    f = urlopen(url)
    
    # info().get_content_charset() : Http 헤더를 기반으로 인코딩 형식 추출 
    encoding = f.info().get_content_charset(failobj = 'utf-8')
    
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    
    return html

### 2. 스크래핑

In [13]:
def scrape(html):
    
    # return 값 : 도서(dictionary) 리스트 -> dataframe
    
    books = []
    
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL): #re.DOTALL : multi lines scan
        
        url = re.search(r'<a href="(.*)?">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url 
        
        title = re.sub(r'<.*?>','', partial_html)
        title = unescape(title) # html.unescape() : return a html string 
        
        books.append(pd.DataFrame({'url':[url],
                                  'title':[title]}))
    
    return pd.concat(books)

### 3. 데이터 저장 

In [20]:
def save(db_path, books):
    
    with sqlite3.connect(os.path.join('.', db_path)) as con : 
        
        try:
            books.to_sql(name = 'BOOKS_INFO', con=con, index = False, if_exists ='append')
            #if_exists : {'fail', 'replace', 'append'} default : fail
            
        except Exception as e:
            print(str(e))
            
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    
    return df #db의 table -> dataframe
            

#### 도서 정보 추출 실습

In [11]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [14]:
df = scrape(html)

In [15]:
df.reset_index(drop = True, inplace = True)

In [18]:
df.head()

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 1권
1,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 2권
2,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 파이썬(2판)
3,http://www.hanbit.co.kr/store/books/look.php?p...,퀀트 전략을 위한 인공지능 트레이딩
4,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 딥러닝


In [21]:
df2 = save('books.db', df)

In [23]:
df2.head()

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 1권
1,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 2권
2,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 파이썬(2판)
3,http://www.hanbit.co.kr/store/books/look.php?p...,퀀트 전략을 위한 인공지능 트레이딩
4,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 딥러닝


저장된 도서 정보 DB 가져오기 

In [24]:
db_name = 'books.db'
table_name = 'BOOKS_INFO'


with sqlite3.connect(db_name) as con:
    try :
        query = 'SELECT * FROM {}'.format(table_name)
        df = pd.read_sql(query, con = con)
        
    except Exception as e:
        print(str(e))
        
    print(df.head())

                                                 url                title
0  http://www.hanbit.co.kr/store/books/look.php?p...      청소년 인문학 수업 - 1권
1  http://www.hanbit.co.kr/store/books/look.php?p...      청소년 인문학 수업 - 2권
2  http://www.hanbit.co.kr/store/books/look.php?p...      처음 시작하는 파이썬(2판)
3  http://www.hanbit.co.kr/store/books/look.php?p...  퀀트 전략을 위한 인공지능 트레이딩
4  http://www.hanbit.co.kr/store/books/look.php?p...          처음 시작하는 딥러닝
